# CAO Points Analysis Using Pandas 

## Fundamentals of Data Analysis Assignment

### Andrew Walker - G00398788@gmit.ie

![https://pandas.pydata.org](https://pandas.pydata.org/docs/_static/pandas.svg)

In [1]:
# Convenient HTTP requests.
#import requests as rq

# Regular expressions.
#import re

# Dates and times.
#import datetime as dt

# Data frames.
#import pandas as pd

# For downloading.
#import urllib.request as urlrq

# Ian's example up to Excel Files and PDFs week

In [2]:
# Regular expressions.
import re

# Convenient HTTP requests.
import requests as rq

# Dates and times
import datetime as dt

# Data frames
import pandas as pd

# FOr downloading
import urllib.request as urlrq

In [3]:
# get the current date and time
now = dt.datetime.now()

# format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

<br>

## 2021 Points

http://www2.cao.ie/points/l8.php

***


In [4]:
# Fetch the CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')
# Have a look
resp

<Response [200]>

### Save original data set

In [5]:
# Create a file path for the original data
pathhtml = 'data/cao2021_LvL8_csv_' + nowstr + '.html'

<br>

** Error on server 

Technically, the server says we should decode as per:
            Content-Type: text/html; charset=iso-8859-1
However, one line uses \x96 which isn't defined in in iso-8859-1.
Therefore using similar decoding standard cp1252 which includes \x96.

In [6]:
# The server uses the wrong encoding, fix it
original_encoding = resp.encoding
# change to cp1252
resp.encoding = 'cp1252'

In [7]:
# Save the original html file
with open(pathhtml, 'w') as f:
    f.write(resp.text)

In [8]:
# Compile the regular expression for matching lines
# re_course = re.compile(r'([A-Z]{2}[0-9]{3}) (.*)([0-9]{3})(\*?) *')
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

### Loop through the lines of the response
***

In [9]:
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
    random = ''
    if s[-1] == '*':
        random = '*'
    points = ''
    for i in s:
        if i.isdigit():
            points = points + i
    return [points, portfolio, random]
        

In [10]:
# File path for the csv file
path2021 = 'data/cao2021_LvL8_csv_' + nowstr + '.csv'

In [11]:
# Keep track of how many courses we process
no_lines = 0

# OPen the csv file for writing
with open(path2021, 'w') as f:
    # Write a header row
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding
        dline = line.decode('cp1252')
        # match only the lines we want - the ones representing courses
        if re_course.fullmatch(dline):
            # add 1 to the lines counter
            no_lines = no_lines + 1
            # Course code
            course_code = dline[:5]
            # Course title
            course_title = dline[7:57].strip()
            # Round 1 points
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma
            linesplit = [course_code, course_title, course_points[0], course_points[1]] 
            # *********** turn the above line course_points into arrays with 3 things in them (see portfolio bit above)
            # Rejoin the substrings with commas in between
            f.write(','.join(linesplit) + '\n')
        
        
# Print the total number of processed lines        
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


In [12]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

In [13]:
df2021

,code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


<br>

## 2020 Points

***

https://www.cao.ie/index.php?page=points&p=2020&bb=points

In [14]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

<br>

#### Save Original File

In [15]:
# Create a file path for the original data
pathxlxs = 'data/cao2020_' + nowstr + '.xlsx'

In [16]:
# Save original file to disk
urlrq.urlretrieve(url2020, pathxlxs)

('data/cao2020_20211222_081926.xlsx',
 <http.client.HTTPMessage at 0x1e1f8192d30>)

<br>

#### Load Spreadsheet using Pandas

In [17]:
# Download and parse the excel spreadsheet
df2020 = pd.read_excel(url2020, skiprows=10)

In [18]:
df2020


,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Spot check a random row
df2020.iloc[753]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [20]:
#Spot check the last but one row
df2020.iloc[-2]

CATEGORY (i.e.ISCED description)                    Personal services
COURSE TITLE                          Recreation and Sport Management
COURSE CODE2                                                    WD212
R1 POINTS                                                         270
R1 Random *                                                       NaN
R2 POINTS                                                         NaN
R2 Random*                                                        NaN
EOS                                                               270
EOS Random *                                                      NaN
EOS Mid-point                                                     349
LEVEL                                                               8
HEI                                 Waterford Institute of Technology
Test/Interview #                                                  NaN
avp                                                               NaN
v                   

In [21]:
# Create a file path for the pandas data
path2020 = 'data/cao2020_' + nowstr + '.csv'

In [22]:
# Save pandas data frame to disk
df2020.to_csv(path2020)

<br>

## 2019 Points

http://www2.cao.ie/points/lvl8_19.pdf

https://www.cao.ie/index.php?page=points&p=2019&bb=points

***


#### Steps to Reproduce

1. Download original pdf file
2. Open original pdf file in MS Word
3. Save MS Word's converted pdf in docx format
4. Re-save MS Word document for editing
5. Delete headers and footers
6. Delete preamble on page 1
7. Select all and copy
8. Paste into Notepad++
9. Remove HEI name headings and paste onto each course line
10. Delete blank lines
11. Replace double tabs with single tabs
12. Change backticks to apostrophes
13. Remove any extra tabs at end of line


In [23]:
df2019 = pd.read_csv('data/cao2019_20211124_080300_edited.csv', sep='\t')

In [24]:
df2019

,Course Code,INSTITUTION and COURSE,EOS,Mid
Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
Waterford Institute of Technology,WD200,Arts (options),221,296.0
Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
Waterford Institute of Technology,WD211,Creative Computing,275,322.0
Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0


In [26]:
#my work
#create new data frame
#manually edit spreadsheet by using find/replace on the institution name
df2019 = pd.read_csv('data/cao2019_20211124_080300_edited_edited.csv', sep='\t')


ParserError: Error tokenizing data. C error: Expected 1 fields in line 29, saw 3


<br>

## concat and join

In [ ]:
courses2021 = df2021[['code', 'title']]
courses2021

In [ ]:
courses2020 = df2020[['COURSE CODE2', 'COURSE TITLE']]
courses2020.columns = ['code', 'title']
courses2020

In [ ]:
# my attempt to add 2019
courses2019 = df2019[['Course Code', 'INSTITUTION and COURSE']]
courses2019.columns = ['code', 'title']
courses2019

In [ ]:
df2019

In [ ]:
allcourses = pd.concat([courses2021, courses2020], ignore_index=True)
allcourses

In [ ]:
allcourses.sort_values('code')

In [ ]:
allcourses.loc[175]['title']

In [ ]:
allcourses.loc[949]['title']

In [ ]:
# Finds all extra copies of duplicated rows
allcourses[allcourses.duplicated()]

In [ ]:
# Returns a copy of the data frame with the duplicates removed
allcourses.drop_duplicates()

In [ ]:
# Finds all extra copies of duplicated rows
allcourses[allcourses.duplicated(subset=['code'])]

In [ ]:
# Returns a copy of the data frame with the duplicates removed
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [ ]:
allcourses

## Join to the points

In [ ]:
#set index to code column
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

In [ ]:
#set index to code column
allcourses.set_index('code', inplace=True)

In [ ]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

In [ ]:
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

In [ ]:
df2020_r1.set_index('code', inplace=True)
df2020_r1

In [ ]:
# Join 2020 points to all courses
allcourses = allcourses.join(df2020_r1)
allcourses

#### next steps - 

- clean up code and titles etc
- add 2019 data to allcourses dataframe
- do a dataframe for r2 points for all three years
- comparisons 

## My attempt - add 2019 data to all courses DF